# Testing notebook

## Working the CSVs and Excel sheets

In [1]:
# Exploring the 'training' dataset
import pandas as pd
df = pd.read_csv("datasets/txt2sql_alerce_train_v3_1.csv")
df.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\r\n-- mjd date for December = 59914.0\r\n-- m...,\r\n-- Super Nova (SNe) is a large explosion t...,"\r\nSELECT\r\n object.oid, probability.clas...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\r\nSELECT\r\n sq1.oid, sq1.probability as SN...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\r\n-- mjd date for September 01 = 60188.0\r\n...,\r\n-- A fast riser is defined as an object wh...,"\r\nSELECT\r\n sq.oid, sq.probability, sq.c...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\r\n-- mjd date for the start of the year 2019...,0,"\r\nSELECT\r\n sq.oid, sq.fid, sq.dmdt_first,...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\r\n-- mjd date for February 01 = 59976.0\r\n-...,0,"\r\nWITH catalog ( source_id, ra, dec) AS (\r\...",advanced,spatial,simple,NaN,NaN,NaN


In [2]:
df[df["req_id"] == 13]["request"].item()

'Give me all the SNe that were first detected between december first 2022 and september first 2023. Return the probability class, the last and the first detection date and the oids of the objects.'

In [3]:
df[df["request"] == "Find at most 10 cases where the feature called 'LinearTrend' is larger than 5 and their ZTF objects have a difference between last and first detection date larger than 1 day. Return the following columns. For the ZTF objects: ZTF identifier and number of detections; for their light curve classifier data at ranking 1: classifier version, class and probability; for the 'LinearTrend' feature: value and filter identifier"]["external_knowledge"].item()

"Since no feature version is specified, the combination between ZTF oid and fid can be non-unique in the 'feature' table output"

In [4]:
df_test = pd.read_csv("datasets/txt2sql_alerce_test_v3_1.csv")
df_test.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,8,Query all objects that were first classified a...,"['probability', 'object']",\r\n-- mjd date for august 17 = 60173.0\r\n-- ...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n object\r\nI...,simple,object,none,NaN,NaN,NaN
1,27,"Return the oids, meanra, meandec, ndet, firstm...","['probability', 'object']",\r\n-- object.ndet represents the total number...,NaN,"\r\nSELECT\r\n object.oid, object.meanra, o...",simple,object,simple,NaN,NaN,NaN
2,17,Return the objects classified as AGN that show...,"['probability', 'object', 'detection']",\r\n-- firstmjd represents the time of the fir...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n (\r\nSELECT *\r...,advanced,object,tree,NaN,NaN,NaN
3,37,"""Query the top 200 objects classified SNIa acc...",['probability'],NaN,NaN,\r\nSELECT\r\n *\r\nFROM\r\n probability\r...,simple,object,none,NaN,NaN,NaN
4,34,Given this list of oids ['ZTF17aaadpsi' 'ZTF19...,['feature'],\r\n-- feature.name can be 'Amplitude'\r\n-- f...,\r\n-- Multiband_period: Period obtained using...,\r\nSELECT\r\n *\r\nFROM\r\n feature\r\n...,simple,other,none,NaN,NaN,NaN


In [5]:
# Reading the excel file
excel = pd.read_excel("datasets/SQLusecases_alerce.xlsx", 
                      sheet_name="examples_alerce_usecasesV3_1")

# Cleaning the excel a bit
excel = excel.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
excel.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,Set,python_format
0,0,Get objects that are likely to be YSOs (possib...,"['probability', 'feature']",\n-- feature.name can be 'Multiband_period'\n-...,\n-- Multiband_period: Period obtained using t...,"\nSELECT\n oid, probability, value, name, fid...",advanced,object,tree,Train,"sub_query_1='''\nSELECT\n feature.oid, prob_o..."
1,1,Get all the objects classified as AGN with a p...,"['object', 'probability', 'feature', 'magstat']",\n-- object.ndet represents the number of dete...,\n-- Amplitude: Half of the difference between...,"\nSELECT\n sq.oid, sq.value, sq.name, sq.fid ...",advanced,object,tree,Test,\nsub_query_object='''\nSELECT\n object.oid...
2,2,Give me the objects classified as YSO by their...,['probability'],NaN,NaN,"\nSELECT\n oid, probability\nFROM\n prob...",simple,object,none,Train,"query='''\nSELECT\n oid, probability\nFROM\..."
3,3,Give me the objects classified as YSO by the l...,"['object', 'probability']","\n-- last june in mjd date: [start=60096.0, en...",NaN,\nSELECT\n *\nFROM\n probability\nWHERE\...,simple,object,simple,Train,query=f'''\nSELECT\n *\nFROM\n probabili...
4,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,NaN,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,Train,# objects classified as SN II with probability...


## Tests

### Gold values and query to test

In [9]:
# Select a query
query = "Give me all the SNe that were first detected between december first 2022 and september first 2023. Return the probability class, the last and the first detection date and the oids of the objects."

# Check if the query is in the excel file
if query in excel["request"].to_list():
    # Obtain the gold SQL query and Python query
    sql_gold = excel[excel["request"] == query]["gold_query"].item()
    python_gold = excel[excel["request"] == query]["python_format"].item()

    # Obtain the necessary tables
    gold_tables = excel[excel["request"] == query]["table_info"].item()

    # Print all in orderly fashion
    print("Gold values\n")
    print("Tables needed for the query:")
    print(gold_tables + "\n")
    print("SQL gold query:")
    print(sql_gold)
    print("Python gold query:\n")
    print(python_gold)
    
else:
    # Obtain the gold SQL query
    sql_gold = df[df["request"] == query]["gold_query"].item()

    # Obtain the necessary tables
    gold_tables = df[df["request"] == query]["table_info"].item()

    # Print all in orderly fashion
    print("Gold values\n")
    print("Tables needed for the query:")
    print(gold_tables + "\n")
    print("SQL gold query:")
    print(sql_gold)

Gold values

Tables needed for the query:
['object', 'probability']

SQL gold query:

SELECT
    object.oid, probability.class_name, object.lastmjd, object.firstmjd
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name IN ('SNIa', 'SNIbc', 'SNII', 'SLSN')
    AND probability.ranking = 1
    AND object.firstmjd < 60217.0
    AND object.firstmjd > 59914.0



In [10]:
# Running the gold query
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

resultGold = pd.read_sql_query(sql_gold, con=engine)
resultGold

,oid,class_name,lastmjd,firstmjd
0,ZTF23aamxbpd,SNIa,60120.218900,60100.190417
1,ZTF23aaphtry,SNIa,60147.231875,60116.315278
2,ZTF23aamzkrs,SNIa,60124.189074,60097.242882
3,ZTF23aafwzmi,SNIa,60078.406505,60050.470231
4,ZTF23aackdba,SNII,60073.174653,59992.212407
...,...,...,...,...
4599,ZTF22aagnxdl,SLSN,60733.522118,59967.502882
4600,ZTF23aasfdzw,SLSN,60739.525313,60112.435741
4601,ZTF22aallpha,SNII,60725.144745,60170.496470
4602,ZTF23aaciqgi,SNII,60730.549861,59987.562743


### Trying out the pipelines

#### Singular

In [7]:
from pprint import pprint
from pipeline.eval import run_pipeline
# from testing.tests import engine
# engine.begin()

# Working query
query = "Give me all the SNe that were first detected between december first 2022 and september first 2023. Return the probability class, the last and the first detection date and the oids of the objects."

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "gpt-4o"

# Format for the pipeline
format = "sql"

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Running the pipeline
result, error, total_usage, prompts, gen_query, gen_label = run_pipeline(query, model, max_tokens, size, overlap, quantity, format, direct=False, rag_pipe=False, self_corr=False)
print("-" * 100)
print("Original query:")
print(df[df["request"] == query]["gold_query"].iloc[0])
print("Predicted query:")
print(gen_query)
print("-" * 100)
print("Resulting table:")
print(result)
print("Total usage of the pipeline:")
pprint(total_usage)

# The prompts used will be saved in this file
with open(f"prompts_query_{model}.txt", "w") as f:
    f.write(str(prompts))

Recreated pipeline
Raw response: ```sql
SELECT p.class_name AS probability_class, 
       o.firstmjd AS first_detection_date, 
       o.lastmjd AS last_detection_date, 
       o.oid
FROM object o
WHERE o.firstmjd BETWEEN 59922 AND 60195
AND o.oid IN (
    SELECT oid
    FROM probability
    WHERE class_name IN ('SNIa', 'SNIbc', 'SNII', 'SLSN')
    AND classifier_name = 'lc_classifier'
    AND ranking = 1
);
```

SELECT p.class_name AS probability_class, 
       o.firstmjd AS first_detection_date, 
       o.lastmjd AS last_detection_date, 
       o.oid
FROM object o
WHERE o.firstmjd BETWEEN 59922 AND 60195
AND o.oid IN (
    SELECT oid
    FROM probability
    WHERE class_name IN ('SNIa', 'SNIbc', 'SNII', 'SLSN')
    AND classifier_name = 'lc_classifier'
    AND ranking = 1
)

Running SQL exception in run_query: (psycopg2.errors.UndefinedTable) missing FROM-clause entry for table "p"
LINE 2: SELECT p.class_name AS probability_class, 
               ^

[SQL: 
SELECT p.class_name AS proba

In [8]:
for key in prompts.keys():
    print(f"Prompt {key}:")
    print(prompts[key])
    print()

Prompt Schema Linking:
['object', 'probability', 'detection']

Prompt Classification:

# For the given request, classify it by difficulty as "simple", "medium", or "advanced" based on the next description.

If (Only 1 table is used, OR 2 most common tables (probability, object or magstat TABLES) are used)
OR (No nested-query or JOIN clause is neccesary, OR only one nested-query between 'probability', 'object' or 'magstat' TABLES is required, OR one JOIN between 'probability', 'object' or 'magstat' TABLES):
THEN "label: simple"

If (2 not common tables are used (NOT probability, object, magstat TABLES))
OR (3 most common tables (probability, object and magstat TABLES) are used)
OR (features with only one feature are used)
OR (Need 1 very complex nested-query, OR a very complex JOIN)
OR (Need 2 nested-query, OR 2 JOIN, OR 1 nested-query with 1 JOIN):
THEN "label: medium"

If (2 or more nested query are needed)
OR (If 3 tables or more are used)
OR (If two features from the features table 

In [2]:
def run_pipeline_debug(query, model, rag_pipe=True, format="sql"):
    print("DEBUG: rag_pipe =", rag_pipe)
    print("DEBUG: format =", format)

run_pipeline_debug("some query", "gpt-3", rag_pipe=False, format="var")


DEBUG: rag_pipe = False
DEBUG: format = var


#### Various

In [ ]:
# Advanced queries of the test dataset
queries = ["Return the objects classified as AGN that show variability in the r band, but have no detections or show no variability in the g band. Order the features by their value in ascending order, and the final query by oid.", "Get all the objects classified as AGN with a probability larger than 0.6, with light curves covering at least one year that have at least 20 detections at least in one band, and select those that show variations larger than 1 magnitude in the g band and larger than 0.5 in the r band. The magnitudes need to be the corrected ones. Return their oid, the value, name, version and filter id from the features, mean coordinates, number of detections, maximum and minimum magnitudes, the variations in bands as the difference between the maximum and minimum magnitudes aliasing the column as 'delta_mag'. For the filter id columns, use the alias '[table_name]_fid' using the table name as the alias.", " Get the object identifiers, probabilities in the stamp classifier and light curves (including only non-detections) for objects whose highest probability in the stamp classifier is obtained for class SN, had their first detection in the first 2 days of september, and qualify as fast risers. Also, return the oid, probability, filter ID and the image difference, and filter ID of magstat as 'magstat_fid', with the initial rise estimate. Order the results by the oid and limit the results to 100 rows due to time constraints.", "Find at most 200 ZTF objects that satisfy the following conditions: have a distance to the nearest PS1 source larger than 5 arcsec, and have a probability larger than 0.7 of being in a class other than supernova in the light curve classifier. Return all columns from the 'object', 'probability' and 'ps1_ztf' table for such objects, and order the results by the nearest PS1 source distance.", "Find at most 10 ZTF objects that have a first detection date later than mjd=60310 days, as well as a probability of 1 of being an asteroid in the stamp classifier, ordered by the first detection date. Return all columns from the 'forced_photometry' table for such objects", "Find at most 50 ZTF objects that satisfy the following conditions: have a probability larger than 0.6 of being an AGN in the light curve classifier, and a WISE W1 magnitude between 12 and 13. Return all columns from the 'magstat' table for such objects", "Find at most 30 ZTF objects that have a probability larger than 0.9 of being an asteroid in the stamp classifier version 'stamp_classifier_1.0.4'. Return the following columns: identifier of the ZTF and candidate, distance from the nearest Solar System object, MPC archive magnitude and name. Include also the following columns related to each candidate in the output table: filter identifier, FWHM from SExtractor, number of PS1 calibrators used, and exposure time", "Find at most 10 ZTF objects that satisfy the following conditions: have more than 500 detections, a probability larger than 0.5 of being an asteroid in the stamp classifier version 'stamp_classifier_1.0.4', and a distance from the nearest Solar System object below 2 arcsec. Return the following columns: ZTF identifier, number of detections, first and last detection date, probability, candidate identifier, distance between ZTF object and MPC object, MPC name, detection date and filter identifier. Order the results by the first detection date.", "Find all detections, non-detections and forced photometry points for the ZTF object 'ZTF24aamtvxb'. Return all epochs in the same output table, including the following columns: ZTF identifier, epoch date, filter identifier, isdiffpos flag, detection difference magnitude and its uncertainty, 5-sigma magnitude limit, forced difference magnitude and its uncertainty, and a column named 'table' (that contains the name of the table of origin for each epoch)", "Find at most 30 ZTF objects that satisfy the following conditions: have more than 10 detections, a probability larger than 0.7 of being a bogus in the stamp classifier version 'stamp_classifier_1.0.4', and  a probability larger than 0.9 of being a transient in the light curve classifier top level. Return the following columns: ZTF identifier, number of detections, first detection date, probability in the stamp classifier (name this column 'prob_stamp_bogus'), and probability in the light curve classifier top level (name this column 'prob_lc_transient'). Order the results by the ZTF identifier of the objects classified as bogus in the stamp classifier."]

# Testing
df_test[df_test["difficulty"] == "simple"]["request"].to_numpy()

In [6]:
# Running the gold query
from secret.config import SQL_URL, USER_10, PASS_10
import requests
import sqlalchemy as sa
import csv
from pipeline.eval import run_pipeline

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{USER_10}:{PASS_10}@{params['host']}/{params['dbname']}", poolclass=sa.pool.NullPool)
engine.begin()

# Model to use
model = "gpt-4o"

# Format for the pipeline
format = "sql"
rag = False

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Select a query
queries = df_test[df_test["difficulty"] == "simple"]["request"].to_numpy()

to_csv = []
for i, query in enumerate(queries):
    # Obtain the gold SQL query
    sql_gold = df_test[df_test["request"] == query]["gold_query"].iloc[0]
    try:
        resultGold = pd.read_sql_query(sql_gold, con=engine)
    except Exception as e:
        print(f"Exception occured running SQL gold: {e}")
        print("Skipping this query for the sake of the CSV")
        continue
    
    # Obtain the true label of the query
    true_label = df_test[df_test["request"] == query]["difficulty"].iloc[0]

    print(f"Experimento número: {i+1}")
    result, error, total_usage, prompts, table, label = run_pipeline(query, model, max_tokens, size, 
                                                overlap, quantity, format, False,
                                                rag, True, 10, 3)
    
    # Saving the actual query used in Python
    full_query = None
    if format == "python" and df_test[df_test["request"] == query]["difficulty"].iloc[0] != "simple":
        exec(table, globals())
        
    # Adding extra validation for result
    if result is not None:
        if "oid" not in result.columns:
            resultRows = result[result.columns[0]].to_numpy()
        else:
            resultRows = result["oid"].to_numpy()
    
    if result is not None:
        # Get predicted oid list
        if 'oid' in result.columns:
            oids_pred = result.sort_values(by="oid",axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'oid_catalog' in result.columns:
            oids_pred = result.sort_values(by="oid_catalog",axis=0).reset_index(drop=True)['oid_catalog'].values.tolist()
        elif 'objectidps1' in result.columns:
            oids_pred = result.sort_values(by="objectidps1",axis=0).reset_index(drop=True)['objectidps1'].values.tolist()
        elif 'classifier_name' in result.columns:
            oids_pred = result.sort_values(by="classifier_name",axis=0).reset_index(drop=True)['classifier_name'].values.tolist()
        elif 'count' in result.columns:
            oids_pred = result.sort_values(by="count",axis=0).reset_index(drop=True)['count'].values.tolist()
            
            # check oids for tipical columns names hallucinations
        elif 'ztf_identifier' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'ztf_identifier': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'ztf identifier' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'ztf identifier': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'ztf_oid' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'ztf_oid': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'ztf_id' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'ztf_id': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'object' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'object': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'ztf' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'ztf': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
        elif 'allwise_id' in [col.lower() for col in result.columns.tolist()]:
            # change the column name to 'oid'
            result.rename(columns={'allwise_id': 'oid'}, inplace=True)
            oids_pred = result.sort_values(by='oid',axis=0).reset_index(drop=True)['oid'].values.tolist()
          
    for_list = {
        "exp": 20 + i + 1,
        "request": query,
        "model": model,
        "format": format,
        "rag": rag,
        "label": label,
        "goldLabel": true_label,
        "genQuery": table,
        "genQueryPython": full_query,
        "goldQuery": sql_gold,
        "results": result,
        "resultsCols": result.columns.to_numpy() if result is not None else None,
        "resultsRows": resultRows if result is not None else None,
        "error": error,
        "resultGold": resultGold,
        "resultGoldCols": resultGold.columns.to_numpy(),
        "resultGoldRows": resultGold["oid"].to_numpy() if "oid" in resultGold.columns else resultGold[resultGold.columns[0]].to_numpy(),
        "usage": total_usage,
        "prompts": prompts
    }
    to_csv.append(for_list)

headers = to_csv[0].keys()
file_name = f"experiments/{format}_RAG_{rag}_simple_v2.csv"
with open(file_name, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    
    # Write the header row
    writer.writeheader()
    
    # Write the data rows
    for row in to_csv:
        writer.writerow(row)

Experimento número: 1
Raw response: ```1
-- Step 1: Convert the date range to Modified Julian Date (MJD) format.
-- Assuming August 17, 2024, corresponds to MJD 60100 and August 21, 2024, to MJD 60104.
-- These values are hypothetical and should be replaced with actual MJD values.

-- Step 2: Sub-query to get oids classified as SN by the stamp classifier with probability > 0.5
SELECT DISTINCT oid
FROM probability
WHERE classifier_name = 'stamp_classifier'
  AND class_name = 'SN'
  AND ranking = 1
  AND probability > 0.5
```

```2
-- Step 3: Sub-query to get oids with first classification date between August 17 and August 21, 2024
SELECT oid
FROM object
WHERE firstmjd BETWEEN 60100 AND 60104
```

```3
-- Step 4: Sub-query to get oids with at least two detections
SELECT oid
FROM object
WHERE ndet >= 2
```

```sql
-- Final Query: Combine all conditions to get the desired objects
SELECT *
FROM object
WHERE oid IN (
    -- Sub-query for classification as SN with probability > 0.5
    SELECT

In [7]:
data = pd.read_csv("experiments/simple/sql_RAG_False_simple_v2.csv")
data

,exp,request,model,format,rag,label,goldLabel,genQuery,genQueryPython,goldQuery,results,resultsCols,resultsRows,error,resultGold,resultGoldCols,resultGoldRows,usage,prompts
0,21,Query all objects that were first classified a...,gpt-4o,sql,False,medium,simple,\n-- Final Query: Combine all conditions to ge...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n object\r\nI...,NaN,NaN,NaN,(psycopg2.errors.QueryCanceled) canceling stat...,oid ndethist ncovhist mjdstar...,['oid' 'ndethist' 'ncovhist' 'mjdstarthist' 'm...,[['ZTF23aaygkbn' 'ZTF23aaygkbn']\n ['ZTF23aayg...,"{'Schema Linking': {'input_tokens': 648, 'outp...","{'Schema Linking': ""['object', 'probability', ..."
1,22,"Return the oids, meanra, meandec, ndet, firstm...",gpt-4o,sql,False,simple,simple,"\nSELECT o.oid, o.meanra, o.meandec, o.ndet, o...",NaN,"\r\nSELECT\r\n object.oid, object.meanra, o...",NaN,NaN,NaN,(psycopg2.errors.UndefinedTable) missing FROM-...,oid meanra meandec nde...,['oid' 'meanra' 'meandec' 'ndet' 'firstmjd' 'd...,['ZTF18abcjwgb' 'ZTF18abcjwgb' 'ZTF18abcjwgb' ...,"{'Schema Linking': {'input_tokens': 687, 'outp...","{'Schema Linking': ""['object', 'probability']""..."
2,23,"""Query the top 200 objects classified SNIa acc...",gpt-4o,sql,False,simple,simple,\nSELECT * \nFROM object \nWHERE oid IN (\n ...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n probability\r...,NaN,NaN,NaN,(psycopg2.errors.CardinalityViolation) more th...,oid classifier_name classifi...,['oid' 'classifier_name' 'classifier_version' ...,['ZTF20acobvxk' 'ZTF22aalpfln' 'ZTF21abuyhau' ...,"{'Schema Linking': {'input_tokens': 641, 'outp...","{'Schema Linking': ""['object', 'probability', ..."
3,24,Given this list of oids ['ZTF17aaadpsi' 'ZTF19...,gpt-4o,sql,False,advanced,simple,\n-- Step 1: Sub-query to filter the 'object' ...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n feature\r\n...,oid feature_name feature_v...,['oid' 'feature_name' 'feature_value'],['ZTF17aaadpsi' 'ZTF17aaadpsi' 'ZTF17aaadpsi' ...,NaN,oid name value ...,['oid' 'name' 'value' 'fid' 'version'],['ZTF19aaduodl' 'ZTF18abtriul' 'ZTF18abtriul' ...,"{'Schema Linking': {'input_tokens': 730, 'outp...","{'Schema Linking': ""['feature', 'object']"", 'C..."
4,25,Write a script in PostgreSQL that return objec...,gpt-4o,sql,False,advanced,simple,\n-- Step 1: Filter objects based on date rang...,NaN,"SELECT\r\n object.oid, object.meanra, objec...",oid meanra meandec n...,['oid' 'meanra' 'meandec' 'ndet' 'firstmjd' 'c...,['ZTF21aaqosgt' 'ZTF21aaqosgu' 'ZTF21aaqosil' ...,NaN,oid meanra meandec n...,['oid' 'meanra' 'meandec' 'ndet' 'firstmjd' 'c...,['ZTF21aanqsrs' 'ZTF21aanqjqc' 'ZTF21aanqpxc' ...,"{'Schema Linking': {'input_tokens': 690, 'outp...","{'Schema Linking': ""['object', 'detection', 'p..."
5,26,Return a Sql query to get the predicted class ...,gpt-4o,sql,False,simple,simple,"\nSELECT \n o.oid, \n t.class_name, \n ...",NaN,"\r\nselect oid, class_name, probability from p...",NaN,NaN,NaN,(psycopg2.errors.UndefinedColumn) column t.cla...,oid class_name probabi...,['oid' 'class_name' 'probability'],['ZTF19aceaaun' 'ZTF18abvorbo' 'ZTF17aacbtqq' ...,"{'Schema Linking': {'input_tokens': 648, 'outp...","{'Schema Linking': ""['object', 'probability', ..."
6,27,Which objects were found between June 1st and ...,gpt-4o,sql,False,simple,simple,\nSELECT * \nFROM object \nWHERE oid IN (\n ...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n object INNE...,NaN,NaN,NaN,(psycopg2.errors.InvalidTextRepresentation) in...,oid ndethist ncovhist mjdst...,['oid' 'ndethist' 'ncovhist' 'mjdstarthist' 'm...,[['ZTF18aaxpmrl' 'ZTF18aaxpmrl']\n ['ZTF18aavl...,"{'Schema Linking': {'input_tokens': 633, 'outp...","{'Schema Linking': ""['object', 'probability']""..."
7,28,Find all ZTF objects that lie within 30 arcsec...,gpt-4o,sql,False,simple,simple,"\nSELECT \n 'id_1' AS identifier, \n 354...",NaN,"WITH catalog (id, obj_ra, obj_dec, r_deg) AS (...",NaN,NaN,NaN,"(psycopg2.errors.UndefinedColumn) column ""ra"" ...",id obj_ra obj_dec r_deg ...,['id' 'obj_ra' 'obj_dec' 'r_deg' 'oid' 'ndet' ...,['ZTF18acfqezx' 'ZTF2

In [8]:
data["request"][0]

'Query all objects that were first classified as SN by the stamp classifier between august 17 and august 21, with a probability greater than 0.5 or at least have two detections.'

In [7]:
print(data["genQuery"][0])
print("-----------------")
print(data["goldQuery"][0])


SELECT * FROM object
WHERE oid IN (
    SELECT oid FROM probability
    WHERE classifier_name = 'stamp_classifier'
    AND class_name = 'SN'
    AND ranking = 1
    AND probability > 0.5
    AND date >= '2023-08-17' AND date <= '2023-08-21'
    UNION
    SELECT oid FROM object
    WHERE oid IN (
        SELECT oid FROM detection
        GROUP BY oid
        HAVING COUNT(*) >= 2
    )
)

-----------------

SELECT
    *
FROM
    object
INNER JOIN
    probability
ON
    object.oid = probability.oid
WHERE
    object.firstMJD > 60173.0
    and object.firstMJD < 60177.0
    AND probability.class_name='SN'
    AND probability.classifier_name='stamp_classifier'
    AND probability.ranking=1
    AND (probability.probability>0.5
    OR object.ndet>1)



In [8]:
print(data["genQuery"][31])
print("-----------------")
print(data["goldQuery"][31])


SELECT name, version, classes
FROM taxonomy
WHERE name = 'lc_classifier'

-----------------
SELECT
    *
FROM
    taxonomy
WHERE
    classifier_name = 'lc_classifier'


In [10]:
for d in range(data.shape[0]):
    print(data["error"][d])

(psycopg2.errors.UndefinedColumn) column "date" does not exist
LINE 9:     AND date >= '2023-08-17' AND date <= '2023-08-21'
                ^

[SQL: 
SELECT * FROM object
WHERE oid IN (
    SELECT oid FROM probability
    WHERE classifier_name = 'stamp_classifier'
    AND class_name = 'SN'
    AND ranking = 1
    AND probability > 0.5
    AND date >= '2023-08-17' AND date <= '2023-08-21'
    UNION
    SELECT oid FROM object
    WHERE oid IN (
        SELECT oid FROM detection
        GROUP BY oid
        HAVING COUNT(*) >= 2
    )
)
]
(Background on this error at: https://sqlalche.me/e/20/f405)
nan
(psycopg2.errors.UndefinedColumn) column "probability" does not exist
LINE 13: ORDER BY probability DESC
                  ^

[SQL: 
SELECT * 
FROM object 
WHERE oid IN (
    SELECT oid 
    FROM probability 
    WHERE classifier_name = 'lc_classifier' 
      AND class_name = 'SNIa' 
      AND ranking = 1 
    ORDER BY probability DESC 
    LIMIT 200
)
ORDER BY probability DESC
]
(Backgroun

#### o1 tests

In [ ]:
from pprint import pprint
from pipeline.main import run_pipeline, engine
engine.begin()

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "o1-preview"

# Format for the pipeline
format = "python"

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Running the pipeline
result, total_usage, prompts = run_pipeline(query, model, max_tokens, size, 
                                            overlap, quantity, format, False, 
                                            engine, rag_pipe=True, 
                                            self_corr=True)
print("Resulting table:")
print(result)
print("Total usage of the pipeline:")
pprint(total_usage)

# The prompts used will be saved in this file
with open(f"prompts/examples/prompts_query_{model}.txt", "w") as f:
    f.write(str(prompts))

Tables needed: [object, detection, magstat]
Difficulty: advanced
```python
# Get oids of objects classified as 'SN II' with probability > 0.6
sub_query_probability = f'''
SELECT oid
FROM probability
WHERE classifier_name='lc_classifier'
AND class_name='SN II'
AND ranking=1
AND probability > 0.6
'''

# Get oids of objects with first observation between 2019-01-01 and 2022-12-31
# MJD range from 58484 (2019-01-01) to 59580 (2022-01-01)
sub_query_object = f'''
SELECT oid
FROM object
WHERE firstmjd BETWEEN 58484 AND 59946
'''

# Combine previous subqueries to get oids satisfying both class and date criteria
sub_query_oid1 = f'''
SELECT prob_oids.oid
FROM ({sub_query_probability}) AS prob_oids
INNER JOIN ({sub_query_object}) AS obj_oids
ON prob_oids.oid = obj_oids.oid
'''

# Get oids with more than 30 detections
sub_query_ndet = f'''
SELECT oid
FROM detection
GROUP BY oid
HAVING COUNT(*) > 30
'''

# Combine to get oids satisfying class, date, and number of detections criteria
sub_query_oid2

Exception: Failed again: local variable 'results' referenced before assignment